In [2]:
import numpy as np
import pandas as pd

In [4]:
#'https://filtereddatasets.s3.amazonaws.com/company-x/train.csv'
df = pd.read_csv('train.csv')
df.drop(columns = 'Unnamed: 0',inplace = True)

## EDA

In [5]:
df.shape

(316970, 27)

In [6]:
df.isnull().sum()

loan_amnt                   0
term                        0
int_rate                    0
installment                 0
grade                       0
sub_grade                   0
emp_title               18398
emp_length              14676
home_ownership              0
annual_inc                  0
verification_status         0
issue_d                     0
loan_status                 0
purpose                     0
title                    1397
dti                         0
earliest_cr_line            0
open_acc                    0
pub_rec                     0
revol_bal                   0
revol_util                213
total_acc                   0
initial_list_status         0
application_type            0
mort_acc                30217
pub_rec_bankruptcies      430
address                     0
dtype: int64

## Data Cleaning

In [4]:
# change data types 
df['issue_d'] = pd.to_datetime(df['issue_d'])
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])

In [9]:
# identify columns with missing data
missing_data = df.isnull().sum().reset_index()
missing_data[missing_data[0]>0]

,index,0
6,emp_title,18398
7,emp_length,14676
14,title,1397
20,revol_util,213
24,mort_acc,30217
25,pub_rec_bankruptcies,430


In [10]:
# will not use emp_title to train the model so just fill with N/A right here, and use the title's mean to fill emp_length
df['emp_title'] = df['emp_title'].fillna('N/A')
df['emp_length'] = df['emp_length'].replace('< 1 year','0.5 year')
df['emp_length'] = df['emp_length'].str.extract(r'(\d+\.?\d*)').astype(float)

fill_length = df.groupby('emp_title')['emp_length'].transform('mean')
df['emp_length'] = df['emp_length'].fillna(fill_length)
# fill na once again, in case all titles don't have employ length
df['emp_length'] = df['emp_length'].fillna(df['emp_length'].mean())

In [11]:
# a lot of missing revol utilization is missing when revolving balance is 0, so first fill in those with 0
df[df['revol_util'].isna()][['revol_util','revol_bal']]

,revol_util,revol_bal
2379,NaN,0.0
2919,NaN,123007.0
5089,NaN,0.0
6288,NaN,0.0
8942,NaN,0.0
...,...,...
309013,NaN,0.0
311884,NaN,0.0
313662,NaN,0.0
313877,NaN,0.0


In [12]:
df.loc[(df['revol_bal']==0) & df['revol_util'].isna(),'revol_util'] = 0
# filling the rest of mean 
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].mean())

In [13]:
# fill mort_acc and pub_rec_bankruptcies with their mode respectively which are both 0
df['mort_acc'] = df['mort_acc'].fillna(0)
df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(0)

## Feature Engineering

In [15]:
df['default'] = df['loan_status'].map({'Fully Paid':0,'Charged Off':1})

In [14]:
# Duration variables
# The latested date of loan issue is at December 2016; therefore the dateset is likely to be old 
# we want to calculate how long has the lender had the loan, we will use 1/1/2017 as the reference date
df['loan_duration'] = (pd.to_datetime('2017-01-01') - df['issue_d']).dt.days
df['credit_duration'] = (pd.to_datetime('2017-01-01') - df['earliest_cr_line']).dt.days

# df['zip'] = df['address'].str.extract(r'(\d{5})')
# df['state'] = df['address'].str.extract(r',\s*([A-Z]{2})\s*\d{5}')

In [16]:
# calculate the debt to income ratio after the loan
df['dti_w_loan'] = (df['installment']+(df['dti']/100)*(df['annual_inc']/12))/(df['annual_inc']/12)
# for a person without income, make the ratio to 9999 same as the person's dti
df.loc[df['dti_w_loan'] == np.inf,'dti_w_loan'] = 9999

In [17]:
# Combination variables
df['term+grade'] = df['term'] + '+' + df['grade']
df['term+homeownership'] = df['term'] + '+' + df['home_ownership']
df['term+verificationstatus'] = df['term'] + '+' + df['verification_status']
df['term+purpose'] = df['term'] + '+' + df['purpose']
df['grade+homeownership'] = df['grade'] + '+' + df['home_ownership']
df['grade+verificationstatus'] = df['grade'] + '+' + df['verification_status']
df['grade+purpose'] = df['grade'] + '+' + df['purpose']
df['homeownership+verificationstatus'] = df['home_ownership'] + '+' + df['verification_status']
df['homeownership+purpose'] = df['home_ownership'] + '+' + df['purpose']
df['verificationstatus+purpose'] = df['verification_status'] + '+' + df['purpose']

In [18]:
# Ratio variables 
# We want to find out how does the loan amount,interest rate, dwi compare to the average based on different categories
df['lratio_grade'] = df['loan_amnt']/df.groupby('grade')['loan_amnt'].transform('mean')
df['lratio_subgrade'] = df['loan_amnt']/df.groupby('sub_grade')['loan_amnt'].transform('mean')
df['lratio_purpose'] = df['loan_amnt']/df.groupby('purpose')['loan_amnt'].transform('mean')
df['lratio_verificationstatus'] = df['loan_amnt']/df.groupby('verification_status')['loan_amnt'].transform('mean')
df['lratio_homeownership'] = df['loan_amnt']/df.groupby('home_ownership')['loan_amnt'].transform('mean')
df['lratio_grade+purpose'] = df['loan_amnt']/df.groupby(['grade','purpose'])['loan_amnt'].transform('mean')
df['lratio_grade+term'] = df['loan_amnt']/df.groupby(['grade','term'])['loan_amnt'].transform('mean')
df['lratio_term+purpose'] = df['loan_amnt']/df.groupby(['term','purpose'])['loan_amnt'].transform('mean')
df['lratio_term+homeownership'] = df['loan_amnt']/df.groupby(['term','home_ownership'])['loan_amnt'].transform('mean')
df['lratio_grade+homeownership'] = df['loan_amnt']/df.groupby(['grade','home_ownership'])['loan_amnt'].transform('mean')
df['lratio_grade+verificationstatus'] = df['loan_amnt']/df.groupby(['grade','verification_status'])['loan_amnt'].transform('mean')
df['lratio_purpose+homeownership'] = df['loan_amnt']/df.groupby(['purpose','home_ownership'])['loan_amnt'].transform('mean')
df['lratio_purpose+verificationstatus'] = df['loan_amnt']/df.groupby(['purpose','verification_status'])['loan_amnt'].transform('mean')

df['iratio_grade'] = df['int_rate']/df.groupby('grade')['int_rate'].transform('mean')
df['iratio_subgrade'] = df['int_rate']/df.groupby('sub_grade')['int_rate'].transform('mean')
df['iratio_purpose'] = df['int_rate']/df.groupby('purpose')['int_rate'].transform('mean')
df['iratio_verificationstatus'] = df['int_rate']/df.groupby('verification_status')['int_rate'].transform('mean')
df['iratio_homeownership'] = df['int_rate']/df.groupby('home_ownership')['int_rate'].transform('mean')
df['iratio_grade+purpose'] = df['int_rate']/df.groupby(['grade','purpose'])['int_rate'].transform('mean')
df['iratio_grade+term'] = df['int_rate']/df.groupby(['grade','term'])['int_rate'].transform('mean')
df['iratio_term+purpose'] = df['int_rate']/df.groupby(['term','purpose'])['int_rate'].transform('mean')
df['iratio_term+homeownership'] = df['int_rate']/df.groupby(['term','home_ownership'])['int_rate'].transform('mean')
df['iratio_grade+homeownership'] = df['int_rate']/df.groupby(['grade','home_ownership'])['int_rate'].transform('mean')
df['iratio_grade+verificationstatus'] = df['int_rate']/df.groupby(['grade','verification_status'])['int_rate'].transform('mean')
df['iratio_purpose+homeownership'] = df['int_rate']/df.groupby(['purpose','home_ownership'])['int_rate'].transform('mean')
df['iratio_purpose+verificationstatus'] = df['int_rate']/df.groupby(['purpose','verification_status'])['int_rate'].transform('mean')

df['dratio_grade'] = df['dti_w_loan']/df.groupby('grade')['dti_w_loan'].transform('mean')
df['dratio_subgrade'] = df['dti_w_loan']/df.groupby('sub_grade')['dti_w_loan'].transform('mean')
df['dratio_purpose'] = df['dti_w_loan']/df.groupby('purpose')['dti_w_loan'].transform('mean')
df['dratio_verificationstatus'] = df['dti_w_loan']/df.groupby('verification_status')['dti_w_loan'].transform('mean')
df['dratio_homeownership'] = df['dti_w_loan']/df.groupby('home_ownership')['dti_w_loan'].transform('mean')
df['dratio_grade+purpose'] = df['dti_w_loan']/df.groupby(['grade','purpose'])['dti_w_loan'].transform('mean')
df['dratio_grade+term'] = df['dti_w_loan']/df.groupby(['grade','term'])['dti_w_loan'].transform('mean')
df['dratio_term+purpose'] = df['dti_w_loan']/df.groupby(['term','purpose'])['dti_w_loan'].transform('mean')
df['dratio_term+homeownership'] = df['dti_w_loan']/df.groupby(['term','home_ownership'])['dti_w_loan'].transform('mean')
df['dratio_grade+homeownership'] = df['dti_w_loan']/df.groupby(['grade','home_ownership'])['dti_w_loan'].transform('mean')
df['dratio_grade+verificationstatus'] = df['dti_w_loan']/df.groupby(['grade','verification_status'])['dti_w_loan'].transform('mean')
df['dratio_purpose+homeownership'] = df['dti_w_loan']/df.groupby(['purpose','home_ownership'])['dti_w_loan'].transform('mean')
df['dratio_purpose+verificationstatus'] = df['dti_w_loan']/df.groupby(['purpose','verification_status'])['dti_w_loan'].transform('mean')

In [19]:
df.drop(columns = ['emp_title','issue_d','title','loan_status','earliest_cr_line','address'],inplace = True)

In [20]:
# Categorical variables
col = df.dtypes.reset_index()
list(col[col[0] == 'object']['index'])

['term',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type',
 'term+grade',
 'term+homeownership',
 'term+verificationstatus',
 'term+purpose',
 'grade+homeownership',
 'grade+verificationstatus',
 'grade+purpose',
 'homeownership+verificationstatus',
 'homeownership+purpose',
 'verificationstatus+purpose']

In [22]:
# Need this to use for testing ratios
training_data = df.copy()

In [21]:
#pip install --upgrade scikit-learn
from sklearn.preprocessing import OneHotEncoder

In [23]:
cat_variables = list(col[col[0] == 'object']['index'])

encoder = OneHotEncoder(sparse_output=False)
cat = encoder.fit_transform(df[cat_variables])
encoded_df = pd.DataFrame(cat,columns=encoder.get_feature_names_out(cat_variables))

df = pd.concat([df,encoded_df],axis=1)
df.drop(columns = cat_variables,inplace=True)

## Feature Selection

### Filter

In [24]:
import scipy.stats as sps

In [25]:
# first seperate default and fully paid loans
good = df[df['default'] == 0]
bad = df[df['default'] == 1]

In [26]:
# select Kolmogorov-Smirnov(KS) test for our filter 
# for each variables, we compare its value in the good vs bad dataframes, 
# the greater the difference the higher the score; thus greater ability to predict
KS_scores = {}
for col in df.columns:
    KS_scores[col] = sps.ks_2samp(good[col],bad[col])[0]

KS = pd.DataFrame(list(KS_scores.items()),columns = ['Variable','score'])

In [28]:
# only keep top 30 variables to use for forward selection
filtered = list(KS.sort_values(by = 'score',ascending = False).head(32)['Variable'])
# since loans are either 36 months or 60 months so we only need 1
filtered.remove('term_ 36 months')

### Wrapper

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SequentialFeatureSelector

In [30]:
filtered_df = df[filtered]
X = filtered_df.drop(columns = 'default')
Y = filtered_df['default']

In [31]:
# use a fast decison tree model to select features
dt = DecisionTreeClassifier(max_depth = 5)
# as mentioned in the problem statement, the firm sufferes the most loss with defaulted loans
# therefore, we use recall(% of total defaults caught by our model) as the evaluation metric to minimize defaults
selection = SequentialFeatureSelector(dt,n_features_to_select = 12,scoring = 'recall')
selection.fit(X,Y)

SequentialFeatureSelector(estimator=DecisionTreeClassifier(max_depth=5),
                          n_features_to_select=12, scoring='recall')

In [32]:
final_variables = list([selection.get_feature_names_out()][0])
selection.get_feature_names_out()

array(['iratio_purpose', 'iratio_purpose+homeownership',
       'iratio_purpose+verificationstatus', 'iratio_term+homeownership',
       'dratio_term+homeownership', 'term_ 60 months',
       'dratio_verificationstatus', 'dratio_term+purpose',
       'dratio_purpose+homeownership', 'dti_w_loan', 'dratio_purpose',
       'loan_duration'], dtype=object)

## Model Selection

### Hyperparameter Tuning

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [34]:
X = df[final_variables]
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,stratify=Y)

In [35]:
models = {'DT':DecisionTreeClassifier(),'RF':RandomForestClassifier(),
          'GB':GradientBoostingClassifier(), 'NN':MLPClassifier()}

param = {'DT':{'max_depth':[15],'min_samples_split':[20,10]},
         'RF':{'n_estimators':[5],'max_depth':[15,20],'min_samples_split':[10,20,50],'max_features':['sqrt','log2']},
         'GB':{'n_estimators':[20,30],'max_depth':[20],'min_samples_leaf':[10,20]},
         'NN':{'hidden_layer_sizes':[(12,),(15,)],'activation':['relu','tanh']}}

In [36]:
model_performance = {}
for name, model in models.items():
    GS = GridSearchCV(model, param[name], cv=5, scoring='recall', n_jobs=-1)
    GS.fit(x_train, y_train)
    model_performance[name] = GS.best_estimator_
    print(name,round(GS.best_score_,3))

DT 0.128
RF 0.142
GB 0.125
NN 0.128


In [37]:
model_performance

{'DT': DecisionTreeClassifier(max_depth=15, min_samples_split=10),
 'RF': RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=5),
 'GB': GradientBoostingClassifier(max_depth=20, min_samples_leaf=10, n_estimators=30),
 'NN': MLPClassifier(hidden_layer_sizes=(15,))}

In [1]:
from sklearn.metrics import recall_score
for name, model in model_performance.items():
    # getting probability and setting threshold to identify defaults
    y_prob = model.predict_proba(x_test)[:,1]
    # we know there about 20% of defauls. To be more active on identifying default, flag 25% loans with highest probability as default
    y_pred = (y_prob >= np.percentile(y_prob,75)).astype(int)
    score = recall_score(y_test,y_pred)
    print(name,' model has a score of ',round(score,3))

DT  model has a score of  0.547
DT  model has a score of  0.624
DT  model has a score of  0.694
DT  model has a score of  0.672


In [3]:
# Parameters for our final model
model_performance['GB']

GradientBoostingClassifier(max_depth=20, min_samples_leaf=10, n_estimators=30)


### Final Model

In [39]:
# Train the model on all available data to have the best performance 
final_model = GradientBoostingClassifier(max_depth=20, min_samples_leaf=10, n_estimators=30)
final_model.fit(X,Y)

GradientBoostingClassifier(max_depth=20, min_samples_leaf=10, n_estimators=30)

In [40]:
Y_prob = final_model.predict_proba(X)[:,1]
Y_pred = (Y_prob >= np.percentile(Y_prob,75)).astype(int)
recall_score(Y,Y_pred)
# model catches 87% of defaults

0.86595440909893

## Prediction

In [42]:
test = pd.read_csv('test.csv')

In [5]:
# Make variables
test['term_ 60 months'] = test['term'].map({' 36 months':0,' 60 months':1})
test['issue_d'] = pd.to_datetime(test['issue_d'])
test['loan_duration'] = (pd.to_datetime('2017-01-01') - test['issue_d']).dt.days
test['dti_w_loan'] = (test['installment']+(test['dti']/100)*(test['annual_inc']/12))/(test['annual_inc']/12)
test.loc[test['dti_w_loan'] == np.inf,'dti_w_loan'] = 9999

In [44]:
# Use saved training_data to get values from training for ratio calculation 
test['iratio_purpose'] = test['int_rate']/training_data.groupby('purpose')['int_rate'].transform('mean')
test['iratio_purpose+homeownership'] = test['int_rate']/training_data.groupby(['purpose','home_ownership'])['int_rate'].transform('mean')
test['iratio_purpose+verificationstatus'] = test['int_rate']/training_data.groupby(['purpose','verification_status'])['int_rate'].transform('mean')
test['iratio_term+homeownership'] = test['int_rate']/training_data.groupby(['term','home_ownership'])['int_rate'].transform('mean')

test['dratio_term+homeownership'] = test['dti_w_loan']/training_data.groupby(['term','home_ownership'])['dti_w_loan'].transform('mean')
test['dratio_verificationstatus'] = test['dti_w_loan']/training_data.groupby('verification_status')['dti_w_loan'].transform('mean')
test['dratio_term+purpose'] = test['dti_w_loan']/training_data.groupby(['term','purpose'])['dti_w_loan'].transform('mean')
test['dratio_purpose+homeownership'] = test['dti_w_loan']/training_data.groupby(['home_ownership','purpose'])['dti_w_loan'].transform('mean')
test['dratio_purpose'] = test['dti_w_loan']/training_data.groupby('purpose')['dti_w_loan'].transform('mean')

In [45]:
X_train = test[list(X.columns)]
Y_prob = final_model.predict_proba(X_train)[:,1]
Prediction = (Y_prob >= np.percentile(Y_prob,75)).astype(int)

In [46]:
data = pd.read_csv('test.csv')
data = pd.concat([data,pd.DataFrame(Prediction,columns = ['predictions'])],axis = 1)

In [47]:
data

,Unnamed: 0,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address,predictions
0,0,14000.0,36 months,11.14,459.28,B,B2,Frederick's of Hollywood,10+ years,RENT,...,0.0,35366.0,48.1,23.0,w,INDIVIDUAL,0.0,0.0,"050 Charles Stream Suite 336\r\nNew Jerryland,...",0
1,1,10000.0,36 months,12.99,336.90,B,B5,Parts Sales,3 years,MORTGAGE,...,1.0,6073.0,65.3,12.0,f,INDIVIDUAL,0.0,1.0,"1212 Pitts Oval\r\nMeyermouth, OH 22690",0
2,2,6000.0,36 months,11.67,198.35,B,B4,secretary,10+ years,RENT,...,1.0,4926.0,39.4,49.0,f,INDIVIDUAL,6.0,1.0,"0843 Price Creek Suite 444\r\nThomashaven, WA ...",0
3,3,6500.0,36 months,15.59,227.21,D,D1,Dental Assistant,2 years,RENT,...,0.0,3100.0,50.0,22.0,w,INDIVIDUAL,0.0,0.0,"27041 Angela Forges Suite 641\r\nWelchmouth, O...",1
4,4,9500.0,36 months,12.12,316.09,B,B3,Wake County Public Schools,9 years,RENT,...,0.0,19246.0,57.8,26.0,f,INDIVIDUAL,0.0,0.0,"PSC 4021, Box 3568\r\nAPO AP 22690",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79055,79055,8000.0,36 months,12.12,266.18,B,B3,Gordon Food Service,2 years,MORTGAGE,...,0.0,7138.0,44.1,26.0,f,INDIVIDUAL,2.0,0.0,70230 Rodriguez Roads Apt. 533\r\nNew Carolynt...,0
79056,79056,18500.0,60 months,13.99,430.37,C,C4,IS Service Desk Supervisor,10+ years,MORTGAGE,...,0.0,3245.0,27.7,10.0,w,INDIVIDUAL,4.0,0.0,"1433 Katherine Place\r\nPort Donald, CA 00813",0
79057,79057,20000.0,36 months,8.18,628.39,B,B1,Guidance Counselor,10+ years,OWN,...,0.0,23190.0,64.2,21.0,f,INDIVIDUAL,0.0,0.0,"2239 Ian Causeway Apt. 612\r\nNorth Lisashire,...",0
79058,79058,5750.0,36 months,16.24,202.84,C,C5,Prep Person,10+ years,MORTGAGE,...,1.0,9625.0,73.0,22.0,w,INDIVIDUAL,4.0,1.0,"203 Thomas Plaza Apt. 960\r\nSandraberg, NM 22690",0
